In [1]:

import torch
import torchvision
import torchvision.transforms as transforms


In [4]:
#The output of torchvision datasets are PILImage images of range[0,1].We transform tem to Tensors of normalized range[-1,1]

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

trainset  = torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset    = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool  = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16 * 5 * 5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

In [8]:
net = Net()

#define loss function and optimizer
#use SGD with momentum

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001 ,momentum=0.9)


In [11]:
#Train th network

for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        #Get the input
        inputs, labels = data
        
        #zero the parameter gradients
        optimizer.zero_grad()
        
        #forward + backward + optimize
        outputs = net(inputs)
        loss    = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        
print('Finished Training')
        

[1,  2000] loss: 1.852
[1,  4000] loss: 1.639
[1,  6000] loss: 1.552
[1,  8000] loss: 1.485
[1, 10000] loss: 1.462
[1, 12000] loss: 1.411
Finished Training
[2,  2000] loss: 1.370
[2,  4000] loss: 1.359
[2,  6000] loss: 1.308
[2,  8000] loss: 1.299
[2, 10000] loss: 1.262
[2, 12000] loss: 1.251
Finished Training


In [12]:
#Test the network on the test data
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images,labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)  #ouput the heighest score
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of thr network on the 10000 test images: %d %% '  %  ( 100 * correct / total))

Accuracy of thr network on the 10000 test images: 54 % 


In [14]:
#Output each class's accuracy
class_correct = list(0. for i in range(10))
class_total   = list(0. for i in range(10))

with torch.no_grad():
    for data in testloader:
        images,labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
for i in range(10):
    print('Accuracy of %5s: %2d %%' %  (classes[i], 100 * class_correct[i] / class_total[i] ))

Accuracy of plane: 55 %
Accuracy of   car: 66 %
Accuracy of  bird: 33 %
Accuracy of   cat: 45 %
Accuracy of  deer: 50 %
Accuracy of   dog: 54 %
Accuracy of  frog: 51 %
Accuracy of horse: 69 %
Accuracy of  ship: 74 %
Accuracy of truck: 43 %


In [ ]:
#Train on GPU
device = torch.device("cuda:0" if torch .cuda.is_available() else "cpu")
print(device)

net.to(device)
inputs,labels = inputs.to(device),labels.to(device)